### Explore SemiAMC model by Liu2022
Use Drone vs. Wifi Data

Limit samples of drone data during training

In [1]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

# import custom functions
from helper_functions import *
from latency_helpers import *
from loading_functions import *
from gamutRF.gamutrf_dataset import *

# import the torch packages
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

import torchvision.models as models

import random
import time

### Load Data

In [2]:
label_dirs= {
    'drone': ['/home/ltindall/data/gamutrf-birdseye-field-days/leesburg_field_day_2022_06_15/worker1/',
              '/home/ltindall/data/gamutrf-birdseye-field-days/pdx_field_day_2022_05_26/worker1/gamutrf/'], 
    'wifi_2_4': ['/home/ltindall/data/gamutrf-pdx/07_21_2022/wifi_2_4/'], 
    'wifi_5': ['/home/ltindall/data/gamutrf-pdx/07_21_2022/wifi_5/']
}


sample_secs = 0.02
nfft = 256
batch_size = 8
num_workers = 19
num_epochs = 4
train_val_test_split = [0.75, 0.05, 0.20]
save_iter = 200
eval_iter = 10000
feat = 'iq'

dataset = GamutRFDataset(label_dirs, sample_secs=sample_secs, nfft=nfft, feat=feat)
print(dataset.idx_to_class)

label='drone', 131 files
label='wifi_2_4', 263 files
label='wifi_5', 366 files


100%|███████████████████████████████████████████████████████████████████████████████████████| 366/366 [00:00<00:00, 389.07it/s]


{0: 'drone', 1: 'wifi_2_4', 2: 'wifi_5'}


In [3]:
print('Number of samples:', len(dataset), "Shape of each shape", dataset.__getitem__(0)[0].shape)

Number of samples: 521694 Shape of each shape torch.Size([2, 419430])


In [4]:
### How many samples of each label?
for lb in dataset.unique_labels:
    print(lb, ' has', sum(didx[0]==lb for didx in dataset.idx))

# ### Set propotion of known and unknown data
labeled_splits = {}
labeled_splits['drone'] = [0.999, 0.001] # [number_labelled, number_unlabelled]
labeled_splits['wifi_2_4'] = [1, 0]
labeled_splits['wifi_5'] = [1, 0]

drone  has 144347
wifi_2_4  has 157747
wifi_5  has 219600


In [5]:
### train_val_test_split
# split whole sample numbers
train_val_test_split_wsamp = [None] * len(train_val_test_split)
for i in range(len(train_val_test_split)-1):
    train_val_test_split_wsamp[i] = int(np.ceil(train_val_test_split[i]*len(dataset)))
train_val_test_split_wsamp[-1] = len(dataset) - sum(train_val_test_split_wsamp[:-1])

train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(dataset, 
                                                                                train_val_test_split_wsamp)

# split into known and unknonwn labels
# known_dataset, unknown_dataset = split_knownunknown_by_label(train_dataset, labeled_splits)
# ^ this is taking a long time

uk_split_samps = [int(train_val_test_split_wsamp[0]*ls) for ls in labeled_splits['drone']]
uk_split_samps[-1] = len(train_dataset) - sum(uk_split_samps[:-1])
print('Known vs. Unknonwn Split:', uk_split_samps)

known_data, unknown_data = torch.utils.data.random_split(train_dataset,
                                                         uk_split_samps)


Known vs. Unknonwn Split: [390879, 392]


In [240]:
# get data loaders
batch_size = 8
num_workers = 10
unknown_dataloader = torch.utils.data.DataLoader(unknown_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
known_dataloader = torch.utils.data.DataLoader(known_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [241]:
def split_knownunknown_by_label(dataset, labeled_splits):
    ktypes = ['k', 'u'] # [known, unknown]
    known_inds = []
    unknown_inds = []
    for i, data in tqdm(enumerate(dataset)):
        ss = time.time()
        start = time.time()
        lbl = dataset.dataset.idx_to_class[int(data[1].numpy())]
        lbl_time = time.time()-start
#         print('labl_time:', lbl_time*1e6)
        
        # choose whether to keep label based on set split
        start = time.time()
        ch = random.choices(ktypes, weights=labeled_splits[lbl],k=1)
        ch_time = time.time()-start
#         print('ch_time:', ch_time*1e6)
        
        start = time.time()
        if ch[0] == 'k':
            known_inds.append(i)
        else:
            unknown_inds.append(i)
        app_time = time.time()-start
#         print('append time:', app_time*1e6)
        
        whole_loop = time.time()-ss
        print('whole_loop:', ss*1e6)
        
    known_data = torch.utils.data.Subset(dataset, known_inds)
    unknown_data = torch.utils.data.Subset(dataset, unknown_inds)
    
    return known_data, unknown_data

### 1. self-supervised contrastive pre-training
Train an encoder to map the original radio frequency signal into a lower dimensional representation, maximimize the similiarities between the signals

In [242]:
## Data augmentation
# thetas = [0, np.pi/2, np.pi, 3*np.pi/2]
# n_aug = 2

# ris = [None]*len(unknown_data)
# rjs = [None]*len(unknown_data)

# for each_samp in tqdm(unknown_data):
#     for k in range(n_aug):
#         the = random.sample(thetas,1)
#         c, s = np.cos(the), np.sin(the)
#         Rotm = np.array(((c, -s), (s, c)))
#         aug_samp = np.zeros(each_samp[0].shape)
#         for s in range(aug_samp.shape[1]):
#             aug_samp[:,s] = np.matmul(Rotm.reshape(2,2), each_samp[0][:,s].reshape(2,1)).reshape(-1)
#         if k ==0:
#             ris[k]=aug_samp
#         else:
#             rjs[k]=aug_samp

In [243]:
class ContrastiveEncoder(nn.Module):
    def __init__(self, to_augment=True):
        super(ContrastiveEncoder,self).__init__()
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=128, kernel_size=24)
#         self.lstm1 = nn.LSTM(input_size=419407, num_layers=2, hidden_size=10, dtype=torch.float64, batch_first=True)
#         self.lstm1 = nn.LSTM(419407, 128)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=8)
        self.mpool = nn.MaxPool2d(kernel_size=128)
        self.encode_lin1 = nn.Linear(in_features=3276, out_features=128, bias=False)
        
        self.proj_lin1 = nn.Linear(in_features=128, out_features=128, bias=False)
        self.proj_lin2 = nn.Linear(in_features=128, out_features=64, bias=False)
        
        self.thetas = [0, np.pi/2, np.pi, 3*np.pi/2]
        self.n_aug = 2
        
        self.to_augment = to_augment
        
    def forward(self, x):
        if self.to_augment:
            samp = self.augmentData(x)
        else:
            samp = x
        ri = self.encode(samp)

#         if not toproj:
#             return ri
        zi = self.projectionHead(ri)
        
        return zi
    
    def encode(self,x):
        x = self.conv1(x)
#         print('past conv1:', x.shape)
#         x, hidden = self.lstm1(x)
#         print('past lstm:',len(x))
        x = self.conv2(x)
#         print('past conv2:', x.shape)
        x = self.mpool(x)
        
        x = self.encode_lin1(x)
        x = torch.squeeze(x,1)
        return x
    
        
    def projectionHead(self,x):
        x = self.proj_lin1(x)
        x = nn.ReLU()(x)
        x = self.proj_lin2(x)
        
        return x
        
    def augmentData(self, x):
#         for k in range(n_aug):
        the = random.sample(self.thetas,1)
        c, s = np.cos(the), np.sin(the)
        Rotm = torch.tensor(np.float32([[c, -s], [s, c]]))
        Rotm = Rotm.reshape(2,2)
        Rotm = Rotm.cuda()
        aug_samp = Rotm.matmul(x) 
#         torch.matmul(Rotm.reshape(2,2), x[:,s].reshape(2,1))
#         aug_samp = np.zeros(x.shape)
#         for s in range(aug_samp.shape[1]):
#             aug_samp[:,s] = torch.matmul(torch.tensor(Rotm.reshape(2,2)), x[:,s].reshape(2,1))
        return torch.tensor(aug_samp)

In [244]:
a = torch.tensor(((1,2),(2,3)))
a.dtype
a = dataset.__getitem__(0)
a[0].shape

torch.Size([2, 419430])

In [245]:
model = ContrastiveEncoder(to_augment=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [246]:
samp = train_dataset.__getitem__(0)[0]
samp = torch.unsqueeze(samp, dim=0)

samp = samp.to(device)
print(samp.shape)
out = model.forward(samp)

print(out.shape)

torch.Size([1, 2, 419430])
torch.Size([1, 64])


/tmp/ipykernel_87453/1250016412.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(aug_samp)


In [247]:
for name, p in model.named_parameters():
    print('lah')
    if "proj" in name: # freeze the project layers
        print(name)
        p.requires_grad = False
    else:
        p.requires_grad = True

lah
lah
lah
lah
lah
lah
proj_lin1.weight
lah
proj_lin2.weight


In [248]:
# class ContrastiveLoss(T.nn.Module):
#     def __init__(self, m=2.0):
#     super(ContrastiveLoss, self).__init__()  # pre 3.3 syntax
#     self.m = m  # margin or radius

#   def forward(self, y1, y2, d=0):
#     # d = 0 means y1 and y2 are supposed to be same
#     # d = 1 means y1 and y2 are supposed to be different
    
#     euc_dist = Torch.nn.functional.pairwise_distance(y1, y2)

#     if d == 0:
#       return T.mean(T.pow(euc_dist, 2))  # distance squared
#     else:  # d == 1
#       delta = self.m - euc_dist  # sort of reverse distance
#       delta = T.clamp(delta, min=0.0, max=None)
#       return T.mean(T.pow(delta, 2))  # mean over all rows

In [249]:
v1 = torch.rand(3,3)
print(v1)
v1[:,0]
# print()

tensor([[0.4022, 0.5813, 0.0727],
        [0.0451, 0.6005, 0.5208],
        [0.1330, 0.4806, 0.6246]])


tensor([0.4022, 0.0451, 0.1330])

In [250]:
torch.mean(v1)

tensor(0.3845)

In [251]:
def contrastive_loss(v1, v2):
    tau = 2
    Ls = torch.zeros((len(v1), len(v2)))
    sims_exp = torch.zeros((len(v1), len(v2)))
    for i in range(len(v1)):
        for j in range(len(v2)):
            sims_exp[i][j] = torch.exp(sim(v1[:,i],v2[:,j])/tau)
    
    for i in range(len(v1)):
        for j in range(len(v2)):
            num = sims_exp[i,j]
            i_row = sims_exp[i,:]
#             print(num)
            denom = torch.sum(i_row) - sims_exp[i][i]
#             print(denom)
            Ls[i][j] = num/denom
    
    return torch.mean(Ls)
            
            
def sim(a,b):
    return torch.dot(a, b)/(torch.norm(b)*torch.norm(a))

In [252]:
output_2.shape
output_1.shape
contrastive_loss(output_1, output_2)

tensor(0.1388, grad_fn=<MeanBackward0>)

In [253]:
# Train the encoder
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs - 1}')

    model.train()  # Set model to training mode

    running_loss = 0.0
    running_corrects = 0
    start = timer()
    for i, (inputs, labels) in enumerate(tqdm(unknown_dataloader)):
        print(f"epoch {epoch}/{num_epochs}, iter {i}/{len(unknown_dataloader)}")
        #print(f"load training data = {timer()-start} seconds")
    
        start = timer() 
        inputs = inputs.to(device)
        labels = labels.to(device)
        #print(f".to(device) = {timer()-start} seconds")
        optimizer.zero_grad()

        start = timer() 
        # 2 augmentations
        output_1 = model.forward(inputs)
        output_2 = model.forward(inputs)
        
        #print(f"inference = {timer()-start} seconds")
        start = timer() 
        loss = contrastive_loss(output_1, output_2)
#         loss = criterion(outputs, labels)
        #print(f"loss = {timer() - start} seconds")
#         _, preds = torch.max(outputs, 1)
#         correct = torch.sum(preds == labels.data)
        print(f"loss={loss.item()}")

        start = timer() 
        loss.backward()
        optimizer.step()
        #print(f"backward and step = {timer()-start} seconds")

#         # statistics
#         running_loss += loss.item() * inputs.size(0)
#         running_corrects += torch.sum(preds == labels.data)
#         start = timer() 
        
#         if (i+1)%save_iter == 0: 
#             model_path = f"resnet18_{str(0.02)}_{epoch}_current.pt"
#             torch.save(model.state_dict(), model_path)
#         if (i+1)%eval_iter == 0: 
#             model.eval()
#             predictions = []
#             labels = []
#             with torch.no_grad():
#                 for j,(data,label) in enumerate(validation_dataloader): 
#                     print(f"validating {j}/{len(validation_dataloader)}")

#                     data = data.to(device)
#                     label = label.to(device)
#                     out = model(data)

#                     _, preds = torch.max(out, 1)
#                     predictions.append(preds.item())
#                     labels.append(label.item())
#                     correct = preds == label.data
#             disp = ConfusionMatrixDisplay.from_predictions(labels, predictions, display_labels=list(dataset.class_to_idx.keys()), normalize='true')
#             disp.figure_.savefig(f"confusion_matrix_{epoch}_{i}.png")
#             model.train()
#     epoch_loss = running_loss / (len(train_dataloader)*batch_size)
#     epoch_acc = running_corrects.double() / (len(train_dataloader)*batch_size)

#     print(f'Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
#     model_path = f"resnet18_{str(0.02)}_{epoch}.pt"
#     torch.save(model.state_dict(), model_path)


Epoch 0/3


  0%|                                                                                                   | 0/49 [00:00<?, ?it/s]

epoch 0/4, iter 0/49


/tmp/ipykernel_87453/1250016412.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(aug_samp)
  0%|                                                                                                   | 0/49 [00:12<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 1.60 GiB (GPU 0; 23.65 GiB total capacity; 20.37 GiB already allocated; 311.06 MiB free; 20.87 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# freeze the encoder layer (flip the layers requiring grad)
for name, param in model.named_parameters():
    param.requires_grad = not param.requires_grad
    print(param.requires_grad)

In [195]:
## Run again with actual labels
model.to_augment = False
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = torch.nn.CrossEntropyLoss()
for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs - 1}')

    model.train()  # Set model to training mode

    running_loss = 0.0
    running_corrects = 0
    start = timer()
    for i, (inputs, labels) in enumerate(tqdm(known_dataloader)):
        print(f"epoch {epoch}/{num_epochs}, iter {i}/{len(unknown_dataloader)}")
        #print(f"load training data = {timer()-start} seconds")
    
        start = timer() 
        inputs = inputs.to(device)
        labels = labels.to(device)
        #print(f".to(device) = {timer()-start} seconds")
        optimizer.zero_grad()

        start = timer() 
        # 2 augmentations
        outputs = model.forward(inputs)
#         output_2 = model.forward(inputs)
        
        #print(f"inference = {timer()-start} seconds")
        start = timer() 
#         loss = contrastive_loss(output_1, output_2)
        loss = criterion(outputs, labels)
        #print(f"loss = {timer() - start} seconds")
        _, preds = torch.max(outputs, 1)
        correct = torch.sum(preds == labels.data)
        print(f"loss={loss.item()}, accuracy={correct/len(preds)}")

        start = timer() 
        loss.backward()
        optimizer.step()
        
        #print(f"backward and step = {timer()-start} seconds")

        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        start = timer() 
        
        if (i+1)%save_iter == 0: 
            model_path = f"resnet18_{str(0.02)}_{epoch}_current.pt"
            torch.save(model.state_dict(), model_path)
        if (i+1)%eval_iter == 0: 
            model.eval()
            predictions = []
            labels = []
            with torch.no_grad():
                for j,(data,label) in enumerate(validation_dataloader): 
                    print(f"validating {j}/{len(validation_dataloader)}")

                    data = data.to(device)
                    label = label.to(device)
                    out = model(data)

                    _, preds = torch.max(out, 1)
                    predictions.append(preds.item())
                    labels.append(label.item())
                    correct = preds == label.data
            disp = ConfusionMatrixDisplay.from_predictions(labels, predictions, display_labels=list(dataset.class_to_idx.keys()), normalize='true')
            disp.figure_.savefig(f"confusion_matrix_{epoch}_{i}.png")
            model.train()
    epoch_loss = running_loss / (len(train_dataloader)*batch_size)
    epoch_acc = running_corrects.double() / (len(train_dataloader)*batch_size)

    print(f'Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
#     model_path = f"resnet18_{str(0.02)}_{epoch}.pt"
#     torch.save(model.state_dict(), model_path)

Epoch 0/3


  0%|                                                                                    | 1/48860 [00:08<121:37:19,  8.96s/it]

epoch 0/4, iter 0/49
loss=4.231062889099121, accuracy=0.0


  0%|                                                                                     | 2/48860 [00:14<96:23:10,  7.10s/it]

epoch 0/4, iter 1/49
loss=4.200510025024414, accuracy=0.0
epoch 0/4, iter 2/49


...

loss=3.933039665222168, accuracy=0.625
epoch 0/4, iter 28/49
loss=3.972670078277588, accuracy=0.375
epoch 0/4, iter 29/49
loss=4.049076080322266, accuracy=0.0
epoch 0/4, iter 30/49


  0%|                                                                                     | 31/48860 [00:36<5:25:34,  2.50it/s]

loss=3.9076881408691406, accuracy=0.625


  0%|                                                                                    | 31/48860 [00:39<17:21:04,  1.28s/it]


KeyboardInterrupt: 

In [197]:
labels

tensor([1, 2, 1, 2, 2, 2, 1, 2], device='cuda:0')

In [26]:
samp_tens = torch.tensor(samp).double()

testConv = nn.Conv1d(in_channels=2, out_channels=32, kernel_size=24)
testConv = testConv.double()

testConv(samp_tens)

tensor([[  541.4630,  -335.9107,  -792.8466,  ...,   866.8007,   699.7957,
          -293.5539],
        [  371.3034,  -302.2969,  -708.7323,  ...,   -75.9819,   546.5313,
           -82.0814],
        [ -332.8842,   183.2735,   626.5565,  ...,   -50.6615,   305.0295,
           628.0556],
        ...,
        [ -382.3249,   478.9416,   797.1793,  ...,   232.8405, -1127.6390,
          -562.7354],
        [  878.6133,  1000.7667,   377.3687,  ...,   498.9463,   260.6467,
           523.4501],
        [  504.7724,   578.6680,   181.0450,  ...,  -439.6437,   758.4425,
          -581.2416]], dtype=torch.float64, grad_fn=<SqueezeBackward1>)

In [22]:
samp_tens

tensor([[ 1247.,   287.,  1263.,  ..., -1870.,  -832.,  1423.],
        [-1599.,   351.,   -48.,  ...,  -351.,  -928.,   512.]],
       dtype=torch.float64)

### 2. map labelled signals to their representation in the low dimensional space
Simple linear layer usually works well

In [15]:
## DEBUG
labeled_filenames = dataset.labeled_files(label_dirs)
idx = []

for label, valid_files in labeled_filenames.items(): 
    for i,full_filename in enumerate(tqdm(valid_files)): 
        print(full_filename)
        idx_filename = full_filename+f"_{str(sample_secs)}.npy"
        print('IDXFILE:',idx_filename)
        if os.path.exists(idx_filename): 
            start = timer()
            file_idx = np.load(idx_filename).tolist()
            print('file idx when exists is:', file_idx)
            print(f"loading {idx_filename}; {i}/{len(valid_files)} time = {timer()-start}")
        if i>0:
            break

label='drone', 131 files
label='wifi_2_4', 263 files
label='wifi_5', 366 files


  1%|▍                                                   | 1/131 [00:00<00:00, 132.88it/s]


/home/ltindall/data/gamutrf-birdseye-field-days/leesburg_field_day_2022_06_15/worker1/gamutrf_recording_ettus_directional-split_gain70_1655312998_5735000000Hz_20971520sps.s16.zst
IDXFILE: /home/ltindall/data/gamutrf-birdseye-field-days/leesburg_field_day_2022_06_15/worker1/gamutrf_recording_ettus_directional-split_gain70_1655312998_5735000000Hz_20971520sps.s16.zst_0.02.npy
file idx when exists is: [['drone', 'data/gamutrf-birdseye-field-days/leesburg_field_day_2022_06_15/worker1/gamutrf_recording_ettus_directional-split_gain70_1655312998_5735000000Hz_20971520sps.s16.zst', '0'], ['drone', 'data/gamutrf-birdseye-field-days/leesburg_field_day_2022_06_15/worker1/gamutrf_recording_ettus_directional-split_gain70_1655312998_5735000000Hz_20971520sps.s16.zst', '1677720'], ['drone', 'data/gamutrf-birdseye-field-days/leesburg_field_day_2022_06_15/worker1/gamutrf_recording_ettus_directional-split_gain70_1655312998_5735000000Hz_20971520sps.s16.zst', '3355440'], ['drone', 'data/gamutrf-birdseye-fiel

  0%|▏                                                   | 1/263 [00:00<00:01, 163.27it/s]


/home/ltindall/data/gamutrf-pdx/07_21_2022/wifi_2_4/gamutrf_recording_ettus_directional_gain70_1658454977_2420000000Hz_20971520sps.s16.zst
IDXFILE: /home/ltindall/data/gamutrf-pdx/07_21_2022/wifi_2_4/gamutrf_recording_ettus_directional_gain70_1658454977_2420000000Hz_20971520sps.s16.zst_0.02.npy
file idx when exists is: [['wifi_2_4', 'data/gamutrf-pdx/07_21_2022/wifi_2_4/gamutrf_recording_ettus_directional_gain70_1658454977_2420000000Hz_20971520sps.s16.zst', '0'], ['wifi_2_4', 'data/gamutrf-pdx/07_21_2022/wifi_2_4/gamutrf_recording_ettus_directional_gain70_1658454977_2420000000Hz_20971520sps.s16.zst', '1677720'], ['wifi_2_4', 'data/gamutrf-pdx/07_21_2022/wifi_2_4/gamutrf_recording_ettus_directional_gain70_1658454977_2420000000Hz_20971520sps.s16.zst', '3355440'], ['wifi_2_4', 'data/gamutrf-pdx/07_21_2022/wifi_2_4/gamutrf_recording_ettus_directional_gain70_1658454977_2420000000Hz_20971520sps.s16.zst', '5033160'], ['wifi_2_4', 'data/gamutrf-pdx/07_21_2022/wifi_2_4/gamutrf_recording_ettus_d

  0%|▏                                                   | 1/366 [00:00<00:02, 136.02it/s]

/home/ltindall/data/gamutrf-pdx/07_21_2022/wifi_5/gamutrf_recording_ettus_directional_gain70_1658467088_5240000000Hz_20971520sps.s16.zst
IDXFILE: /home/ltindall/data/gamutrf-pdx/07_21_2022/wifi_5/gamutrf_recording_ettus_directional_gain70_1658467088_5240000000Hz_20971520sps.s16.zst_0.02.npy
file idx when exists is: [['wifi_5', 'data/gamutrf-pdx/07_21_2022/wifi_5/gamutrf_recording_ettus_directional_gain70_1658467088_5240000000Hz_20971520sps.s16.zst', '0'], ['wifi_5', 'data/gamutrf-pdx/07_21_2022/wifi_5/gamutrf_recording_ettus_directional_gain70_1658467088_5240000000Hz_20971520sps.s16.zst', '1677720'], ['wifi_5', 'data/gamutrf-pdx/07_21_2022/wifi_5/gamutrf_recording_ettus_directional_gain70_1658467088_5240000000Hz_20971520sps.s16.zst', '3355440'], ['wifi_5', 'data/gamutrf-pdx/07_21_2022/wifi_5/gamutrf_recording_ettus_directional_gain70_1658467088_5240000000Hz_20971520sps.s16.zst', '5033160'], ['wifi_5', 'data/gamutrf-pdx/07_21_2022/wifi_5/gamutrf_recording_ettus_directional_gain70_165846

In [16]:
np.load(idx_filename).tolist()

[['wifi_5',
  'data/gamutrf-pdx/07_21_2022/wifi_5/gamutrf_recording_ettus_directional_gain70_1658464188_5780000000Hz_20971520sps.s16.zst',
  '0'],
 ['wifi_5',
  'data/gamutrf-pdx/07_21_2022/wifi_5/gamutrf_recording_ettus_directional_gain70_1658464188_5780000000Hz_20971520sps.s16.zst',
  '1677720'],
 ['wifi_5',
  'data/gamutrf-pdx/07_21_2022/wifi_5/gamutrf_recording_ettus_directional_gain70_1658464188_5780000000Hz_20971520sps.s16.zst',
  '3355440'],
 ['wifi_5',
  'data/gamutrf-pdx/07_21_2022/wifi_5/gamutrf_recording_ettus_directional_gain70_1658464188_5780000000Hz_20971520sps.s16.zst',
  '5033160'],
 ['wifi_5',
  'data/gamutrf-pdx/07_21_2022/wifi_5/gamutrf_recording_ettus_directional_gain70_1658464188_5780000000Hz_20971520sps.s16.zst',
  '6710880'],
 ['wifi_5',
  'data/gamutrf-pdx/07_21_2022/wifi_5/gamutrf_recording_ettus_directional_gain70_1658464188_5780000000Hz_20971520sps.s16.zst',
  '8388600'],
 ['wifi_5',
  'data/gamutrf-pdx/07_21_2022/wifi_5/gamutrf_recording_ettus_directional_ga

In [18]:
a = np.load(idx_filename)

In [22]:
idx_filename

'/home/ltindall/data/gamutrf-pdx/07_21_2022/wifi_5/gamutrf_recording_ettus_directional_gain70_1658464188_5780000000Hz_20971520sps.s16.zst_0.02.npy'

In [ ]:
## Import Data
main_folder = '/home/ltindall/data/DroneDetect/DroneDetect_V2/' # data path on mp-gpu-desktop06

load_dronedetect_raw(main_folder+sf+'/'+df+'/'+fi, t_seg)

In [ ]:
feat_folder = '../Features/'
feat_name = 'SPEC'
seg_len = 20
n_per_seg = 1024
interferences = ['WIFI','CLEAN','BLUE','BOTH']
output_name = 'drones'
feat_format = 'IMG'

dataset = DroneDetectTorch(feat_folder, feat_name, seg_len, n_per_seg, feat_format,
                                output_name, interferences)